In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import U_config as cfg
import H_grid_set as gs


# GRID INITIALISATION:
m = ccrs.NorthPolarStereo()
GRID = gs.grid_set(m)
GRID.load_grid(cfg.THICKNESS_GRID)
f = plt.figure()
ax = f.add_subplot(1,1,1,projection=m)
ax.set_extent([-180, 180, 65, 90], ccrs.PlateCarree())
Gplot = gs.grid_set(m)
Gplot.set_grid_mn(30, 30,ax) 
Gplot.gridinfo = False
Gplot.get_grid_info(av_ang=False)
GP2Gplot= gs.Gs2Gs(GRID,Gplot,vectors=True)
f.clear()

In [ ]:
def pre_process_variable(sample, ground_truth):
    sample[sample>1] = 1
    sample[sample<0] = 0
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    return sample[:100, :100]

def remove_land(sample, ground_truth):
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    return sample[:100, :100]

def load_year(year):
    truth = []
    for i in range(1, 13):
        truth.append(np.load(f"../data/processed_data/sic/{year}_{f'0{i}' if i<10 else i}.npy"))
    return np.array(truth)

In [ ]:
maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
sic_path = "../data/datasets/post_standard/sic/"

def mask_non_active_region(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    maximum_sic = np.load(sic_path+maximum)
    sample[np.where(maximum_sic<0.15)]=np.nan
    return sample[:100, :100]*100

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1,projection=ccrs.NorthPolarStereo(central_longitude=0))
ax.coastlines(linewidth=0.5,color='k')
ax.add_feature(cfeature.OCEAN)
NSIDC = np.load(f"../data/datasets/post_standard/sic/2019_09.npy")[:100,:100]
NSIDC_v = load_year(2019)
mask = NSIDC_v[8]
NSIDC = pre_process_variable(NSIDC, mask)

#ax.contourf(GRID.xptp, GRID.yptp, NSIDC, 15, cmap=plt.cm.get_cmap("Blues").reversed())
ax.pcolormesh(GRID.xptp, GRID.yptp, mask, vmin=0, vmax=1, cmap=plt.cm.get_cmap("Blues").reversed())
ax.coastlines(linewidth=0.5,color='k')
ax.add_feature(cfeature.LAND)
gls = ax.gridlines(color='C7',lw=1,ls=':',draw_labels=True,rotate_labels=False,ylocs=[70,75,80])
polarCentral_set_latlim((65,90),ax)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import numpy as np
import cartopy.crs as ccrs

# Helper function
# from https://nordicesmhub.github.io/NEGI-Abisko-2019/training/example_NorthPolarStereo_projection.html
def polarCentral_set_latlim(lat_lims, ax):
    ax.set_extent([-180, 180, lat_lims[0], lat_lims[1]], ccrs.PlateCarree())
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)

In [ ]:
EXPERIMENT_ID = 301
NSIDC = load_year(2019)
mask = NSIDC[0]

thickness_predictions = np.load(f"../model/models/{EXPERIMENT_ID}/post_training/non_evidenced_estimates.npy")
baseline_predictions = np.load(f"../model/models/{EXPERIMENT_ID}/post_baseline/non_evidenced_estimates.npy")
#thickness_plots = f"../model/models/{EXPERIMENT_ID}/post_training/"
#baseline_plots = f"../model/models/{EXPERIMENT_ID}/post_baseline/"
#comparitive_plots = f"../model/models/{EXPERIMENT_ID}/comparative"

THICK = []
for i in range(0, 12):
    THICK.append(pre_process_variable(thickness_predictions[i][0], mask))

In [ ]:
fig, axes = plt.subplots(2,6, figsize=(12, 6), constrained_layout=True, subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)})
fig.suptitle(f'Model predictions for 2019', fontsize=12)
map=plt.cm.get_cmap('Spectral').reversed()

titles = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

count = 0
for ax in axes.flat:
    ax.set_extent([-180, 180, 65, 90], ccrs.PlateCarree())
    ax.set_title(titles[count], fontsize=9)
    ax.add_feature(cfeature.LAND)
    ax.coastlines(linewidth=0.2,color='k')
    if count == 10:
        s=ax.pcolormesh(GRID.xptp, GRID.yptp, THICK[count]*100, vmin=0, vmax=100, cmap=plt.cm.get_cmap("Blues").reversed())
    else:
        ax.pcolormesh(GRID.xptp, GRID.yptp, THICK[count]*100, vmin=0, vmax=100, cmap=plt.cm.get_cmap("Blues").reversed())
    #polarCentral_set_latlim((65,90),ax)
    count+=1
    ax.gridlines(draw_labels=False)

cbar = fig.colorbar(s, ax=axes.ravel().tolist(), location='right', shrink=0.4, pad=0.02)
for t in cbar.ax.get_yticklabels():
    t.set_fontsize(8)

plt.show()

In [ ]:
maximums = ["2012_01.npy", "2013_02.npy", "2013_03.npy", "2010_04.npy", "2010_05.npy", "2013_06.npy", "2013_07.npy", "2014_08.npy", "2014_09.npy", "2013_10.npy", "2014_11.npy", "2014_12.npy"]
sic_path = "../data/datasets/post_standard/sic/"

def mask_non_active_region(sample, ground_truth, maximum):
    land = np.where(np.isnan(ground_truth))
    sample[land]= np.nan
    maximum_sic = np.load(sic_path+maximum)
    sample[np.where(maximum_sic<0.15)]=np.nan
    return sample[:100, :100]

NSIDC = load_year(2019)
mask = NSIDC[0]
EXPERIMENT_ID = 401

In [ ]:
U_DRIFT_HERE = U_DRIFT
V_DRIFT_HERE = V_DRIFT

In [ ]:
THICKNESS = []
for i in range(1, 13):
    m = f'{i}' if i > 9 else f'0{i}'
    THICKNESS.append(remove_land(np.load(f), mask))

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(9, 15), constrained_layout=True, subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)})
fig.suptitle(f'CryoSat Freeboard Thickness', fontsize=12)
map=plt.cm.get_cmap('Spectral').reversed()

titles = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

count = 0
for ax in axes.flat:
    ax.set_extent([-180, 180, 65, 90], ccrs.PlateCarree())
    ax.set_title(titles[count], fontsize=11)
    ax.add_feature(cfeature.LAND)
    ax.coastlines(linewidth=0.6,color='k')
    if count == 11:
        s=ax.pcolormesh(GRID.xptp,GRID.yptp,THICKNESS[count], vmin=0, vmax=5, cmap=plt.cm.get_cmap('Spectral').reversed())
    else:
        ax.pcolormesh(GRID.xptp,GRID.yptp,THICKNESS[count], vmin=0, vmax=5, cmap=plt.cm.get_cmap('Spectral').reversed())
    polarCentral_set_latlim((65,90),ax)
    count+=1

cbar = fig.colorbar(s, ax=axes.ravel().tolist(), location='bottom', shrink=0.6, pad=0.02, extend="both")
cbar.set_label("Thickness in (m)", fontsize=9)

for t in cbar.ax.get_yticklabels():
    t.set_fontsize(8)

plt.show()

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(9, 15), constrained_layout=True, subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)})
fig.suptitle(f'NSIDC Pathfinder Drift', fontsize=12)
map=plt.cm.get_cmap('Spectral').reversed()

titles = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

count = 0
for ax in axes.flat:
    ax.set_extent([-180, 180, 65, 90], ccrs.PlateCarree())
    ax.set_title(titles[count], fontsize=11)
    ax.add_feature(cfeature.LAND)
    ax.coastlines(linewidth=0.6,color='k')
    if count == 11:
        s=ax.pcolormesh(GRID.xptp,GRID.yptp,np.hypot(U_DRIFT_HERE[count],V_DRIFT_HERE[count]), vmin=-0.05, vmax=0.15, cmap=plt.cm.get_cmap('RdBu').reversed())
    else:
        ax.pcolormesh(GRID.xptp,GRID.yptp,np.hypot(U_DRIFT_HERE[count],V_DRIFT_HERE[count]), vmin=-0.05, vmax=0.15, cmap=plt.cm.get_cmap('RdBu').reversed())

    ur,vr = GP2Gplot.rg_vecs(U_DRIFT_HERE[count],V_DRIFT_HERE[count])
    ax.quiver(Gplot.xpts,Gplot.ypts,ur,vr,scale = 1.1)
    polarCentral_set_latlim((65,90),ax)
    count+=1

cbar = fig.colorbar(s, ax=axes.ravel().tolist(), location='right', shrink=0.6, pad=0.02, extend="both")
cbar.set_label("Velocity in ()", fontsize=9)

for t in cbar.ax.get_yticklabels():
    t.set_fontsize(8)

plt.show()

In [ ]:
SIC = []
for i in range(1, 13):
    m = f'{i}' if i > 9 else f'0{i}'
    SIC.append(remove_land(np.load(f"../data/processed_data/sic/2019_{m}.npy"), mask))

SIT_IMP = []
SIT = np.load(f"../model/models/301/SIT.npy")
for i in range(1, 13):
    m = f'{i}' if i > 9 else f'0{i}'
    SIT_IMP.append(SIT[i-1][0][:100, :100])

V_DRIFT = []
V = np.load(f"../model/models/301/V_DRIFT.npy")
for i in range(1, 13):
    m = f'{i}' if i > 9 else f'0{i}'
    V_DRIFT.append(V[i-1][0][:100, :100])

U_DRIFT = []
U = np.load(f"../model/models/301/U_DRIFT.npy")
for i in range(1, 13):
    m = f'{i}' if i > 9 else f'0{i}'
    U_DRIFT.append(V[i-1][0][:100, :100])

SIT_ANOM = (np.array(SIC) - np.array(SIT_IMP))[5:9]
U_ANOM = (np.array(SIC) - np.array(U_DRIFT))[5:9]
V_ANOM = (np.array(SIC) - np.array(V_DRIFT))[5:]

In [ ]:
fig, axes = plt.subplots(1,4, figsize=(5, 2), constrained_layout=True, subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)})
titles = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

count = 0
for ax in axes.flat:
    ax.set_extent([-104, -170, 65, 90], ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.coastlines(linewidth=0.6,color='k')
    ax.set_title(titles[count+5], fontsize=11)
    c=ax.pcolormesh(GRID.xptp,GRID.yptp,SIC[count+5]*100, vmin=0, vmax=100, cmap=plt.cm.get_cmap('Blues').reversed())
    count+=1

cbar = fig.colorbar(c, ax=axes.ravel().tolist(), location='bottom', shrink=0.4, pad=0.02)
cbar.set_label("Sea Ice Concentration %", fontsize=9)

for t in cbar.ax.get_yticklabels():
    t.set_fontsize(8)

plt.show()

In [ ]:
fig, axes = plt.subplots(3,4, figsize=(7, 6), constrained_layout=True, subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)})
map=plt.cm.get_cmap('Spectral').reversed()

titles = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

count = 0
for ax in axes.flat:
    ax.set_extent([-104, -170, 65, 90], ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.coastlines(linewidth=0.6,color='k')
    
    if count < 4:
        ax.set_title(titles[count+5], fontsize=11)
        ax.pcolormesh(GRID.xptp,GRID.yptp,SIT_ANOM[count]*100, vmin=-50, vmax=50, cmap=plt.cm.get_cmap('RdBu'))
    elif count < 8:
        ax.pcolormesh(GRID.xptp,GRID.yptp,U_ANOM[count-4]*100, vmin=-50, vmax=50, cmap=plt.cm.get_cmap('RdBu'))
    else:
        if count == 11:
            s=ax.pcolormesh(GRID.xptp,GRID.yptp,V_ANOM[count-8]*100, vmin=-50, vmax=50, cmap=plt.cm.get_cmap('RdBu'))
        else:
            ax.pcolormesh(GRID.xptp,GRID.yptp,V_ANOM[count-8]*100, vmin=-50, vmax=50, cmap=plt.cm.get_cmap('RdBu'))
    
    #polarCentral_set_latlim((65,90),ax)
    count+=1

cbar = fig.colorbar(s, ax=axes.ravel().tolist(), location='bottom', shrink=0.6, pad=0.02, extend="both")
cbar.set_label("Error Residual in %", fontsize=9)

for t in cbar.ax.get_yticklabels():
    t.set_fontsize(8)

plt.show()

In [ ]:
fig, axes = plt.subplots(2,4, figsize=(12, 6), constrained_layout=True, subplot_kw={"projection": ccrs.NorthPolarStereo(central_longitude=0)})
map=plt.cm.get_cmap('plasma').reversed()
EXPERIMENT_ID = 401
titles = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
thickness_epi = np.load(f"../model/models/{EXPERIMENT_ID}/post_training/epistemic.npy")
baseline_epi = np.load(f"../model/models/{EXPERIMENT_ID}/post_baseline/epistemic.npy")

count = 0
base_count = 8
thick_count = 8

for ax in axes.flat:

    ax.set_extent([-180, 180, 65, 90], ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.coastlines(linewidth=0.6,color='k')
    
    if count < 4:    
        ax.set_title(titles[base_count], fontsize=11)
        base = baseline_epi[base_count][0][:100,:100]
        ax.pcolormesh(GRID.xptp,GRID.yptp,base, vmin=0, vmax=3, cmap=plt.cm.get_cmap('Spectral').reversed())
        base_count += 1
    else:
        ax.set_title(titles[thick_count], fontsize=11)
        thick = thickness_epi[thick_count][0][:100,:100]     

        if count == 6:
            s=ax.pcolormesh(GRID.xptp,GRID.yptp,thick, vmin=0, vmax=3, cmap=plt.cm.get_cmap('Spectral').reversed())
        else:
            ax.pcolormesh(GRID.xptp,GRID.yptp,thick, vmin=0, vmax=3, cmap=plt.cm.get_cmap('Spectral').reversed())
        thick_count += 1
        
    count+=1
    polarCentral_set_latlim((65,90),ax)

cbar = fig.colorbar(s, ax=axes.ravel().tolist(), location='right', shrink=0.6, pad=0.02, extend="both")
cbar.set_label("Epistemic Uncertainty", fontsize=9)

for t in cbar.ax.get_yticklabels():
    t.set_fontsize(8)

plt.show()